In [1]:
!pip install selenium

     |████████████████████████████████| 911kB 3.4MB/s 


In [0]:
#네이버 뉴스 댓글 크롤링
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from pprint import pprint
import pandas as pd

In [3]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
#드라이버 연결 인스턴스
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver.implicitly_wait(1)

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [88.1 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelea

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [0]:
def capture_replys(url, excel_name=None):
    # url = 'https://sports.news.naver.com/news.nhn?oid=413&aid=0000096695&m_view=1&sort=LIKE'
    if not '&m_view=1' in url:
        url += '&m_view=1'

    #페이지 이동하기
    print("[접속하기]")
    driver.get(url)

    #더보기 버튼 누르기
    print("[더보기 클릭 중]")
    attemp = 0
    while True:
        try:
            driver.find_element_by_css_selector('span.u_cbox_page_more').click()
            attemp = 0
        except:
            attemp +=1
            if attemp > 5:
                break #5번 이상 시도시 더보기 버튼이 없다고 생각한다.

    #댓글 요소 찾기
    print("[댓글 요소 찾기]")
    replys = driver.find_elements_by_css_selector('ul.u_cbox_list > li.u_cbox_comment')
    print(len(replys))

    #작성자 : span.u_cbox_nick, 댓글 내용 : span.u_cbox_contents
    ##작성자와 댓글 내용 추출 (작성자, 댓글내용)

    print("[댓글 내용 수집]")
    results = []
    del_message = 0


    for reply in replys:
        try:
            #author = reply.find_element_by_css_selector('span.u_cbox_nick').text
            content = reply.find_element_by_css_selector('span.u_cbox_contents').text
            #results.append((author,content))
            results.append((content))
        except:
            del_message += 1 #삭제된 댓글 카운트

    print("삭제된 댓글 갯수 : ",del_message)
    pprint(results)

    #엑셀파일 만들기
    if excel_name:
        print("[전체 댓글 엑셀로 저장]")
        # col = ["작성자","내용"]
        col = ["내용"]
        data_frame = pd.DataFrame(results,columns=col)
        data_frame.to_excel('/content/gdrive/My Drive/Colab Notebooks/DataCrawling/Result/{}.xlsx'.format(excel_name),sheet_name='Sheet1',startrow=0,header=True)
    else:
        print("[전체 댓글 엑셀로 저장하지 않음]")

    #크롬드라이버 창 닫아주기
    time.sleep(3)
    driver.quit()

In [0]:
# 실행,테스트
if __name__ == '__main__':
    capture_replys('https://sports.news.naver.com/news.nhn?oid=413&aid=0000096204','[오피셜] 기성용, 스페인 마요르카 입단')

[접속하기]
[더보기 클릭 중]
[댓글 요소 찾기]
979
[댓글 내용 수집]
